In [1]:
#Imports libraries and loads data into pandas dataframes

import numpy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.io.json import json_normalize
from pandas.tseries import converter as pdtc
from sklearn.svm import SVR
import matplotlib.units as munits
from keras.layers import Dropout
from keras.regularizers import l2
from datetime import datetime
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from sklearn.linear_model import LinearRegression
from scipy import stats
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from functools import reduce
from datetime import datetime
from keras.optimizers import SGD
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import read_csv
from pandas.plotting import scatter_matrix
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Lambda, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras
import json
import os
import sklearn
import numpy
from datetime import datetime
from dateutil.parser import parse
import findspark
findspark.init()
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
import pyspark
pd.set_option('display.max_rows', None)

#Initialize spark session
spark = SparkSession \
        .builder \
        .appName("ensemble_learning_sparktest") \
        .getOrCreate()

def concatenate_json_to_df(directory_path, to_output, folder_output):
    list_json = []
    
    for file in os.listdir(directory_path):
        if file.split(".")[-1] == "json":
            
            with open(directory_path+file, 'r') as f:
            
                # If json file is not empty
                if os.stat(directory_path+file).st_size != 0:
                                
                        
                    # For Observation file type
                    if directory_path.split("/")[-2] == "Observations":
                        dictio = json.load(f)
                                            
                        if 'observation' in dictio:
                            observation = dictio["observation"]
                        
                            for entry in observation:
                                entry['date'] =  file.rstrip(".json")
                                entry['station'] = dictio['station']
                                entry['time'] = datetime.utcfromtimestamp(entry['time']).strftime('%d-%m-%Y %H:%M:%S')
                                list_json.append(entry)
                    
                    
                    # For Analyses file type
                    elif directory_path.split("/")[-2] == "Analyses":
                        dictio = json.load(f)
                        if 'source' in dictio and 'temperature' in dictio:
                            source = list(dictio['source'])
                            temperature = list(dictio['temperature'])
                            
                            dictio['date'] = file.rstrip(".json")
                            dictio.pop('source', None)
                            dictio.pop('temperature', None)
                            
                            entry = dict()
                            for i in range(0, len(source)):
                                dictio[source[i]] = temperature[i]
                                list_json.append(dictio)

                                
                    # For Previsions file type
                    elif "Previsions" in directory_path.split("/"):
                        dictio = json.load(f)
                        
                        latit_r = None
                        long_r = None
                        
                        if "longitude_r" in dictio:
                            long_r = dictio['longitude_r']

                        if "latitude_r" in dictio:
                            latit_r = dictio['latitude_r']
                            
                        if 'forecast' in dictio:
                            forecast = dictio['forecast'] 
                            
                            
                            for i in range(0, len(forecast)):
                                dictio = dict()
                                
                                if long_r != None and latit_r != None:
                                    dictio['latitude_r'] = latit_r
                                    dictio['longitude_r'] = long_r
                                
                                if "temperature" in forecast[i]:
                                    dictio["temperature"] = forecast[i]["temperature"]
                                    
                                if "time" in forecast[i]:
                                    dictio["time"] = datetime.utcfromtimestamp(forecast[i]["time"]).strftime('%d-%m-%Y %H:%M:%S')
                                    
                                if "wind" in forecast[i]:
                                    dictio["wind"] = forecast[i]["wind"]
                                
                                if "wind_dir" in forecast[i]:
                                    dictio["wind_dir"] = forecast[i]["wind_dir"]
                                
                                if "humidity" in forecast[i]:
                                    dictio["humidity"] = forecast[i]["humidity"]
                                
                                if "pressure" in forecast[i]:
                                            dictio["pressure"] = forecast[i]["pressure"]
                                        
                                if "cloud_cover" in forecast[i]:
                                            dictio["cloud_cover"] = forecast[i]["cloud_cover"]

    
                                
                                dictio["date"] = file.rstrip(".json")
                                
                                list_json.append(dictio)
                
                
                else:
                    dictio = dict()
                    dictio["date"] = file.rstrip(".json")
                    list_json.append(dictio)
                                    
    df = pd.DataFrame(list_json)
    df.drop_duplicates()
    df.to_csv(folder_output+"/"+to_output.replace("/", "-")+".csv", sep=";", index=False)
    
    
dirname = "technical_challenge_v9"
if not os.path.exists(dirname):
    os.mkdir(dirname)
    
concatenate_json_to_df('technical_challenge/Asport CAPRED/Previsions/Api-Agro/', "Asport CAPRED/Previsions/Api-Agro", dirname)
# Launch data of Api, Dark and weatherbit forcesaters for the station Asport
path = "technical_challenge_v9/Asport CAPRED-Previsions-"
path_previsions = ["Api-Agro.csv", "Dark Sky.csv", "Weatherbit.csv"]
path_observations = "technical_challenge_v9/Asport CAPRED-Observations.csv"

def get_forecast(path, path_previsions, path_observations):
    list_data = []
    for forecaster in path_previsions:
        dataframe = pd.read_csv(path + forecaster, sep=";")
        dataframe["time"] = pd.to_datetime(dataframe['time'], format='%d-%m-%Y %H:%M:%S')
        dataframe.sort_values(by=['time'])
        dataframe['time'] = dataframe['time'].apply(lambda x: x.replace(minute=0, second=0))
        dataframe = dataframe.groupby(['time']).apply(lambda x : x.iloc[:6].mean())
        dataframe.columns = [x + "_" + forecaster.split('.')[0] for x in dataframe.columns]
        list_data.append(dataframe)   
    
    observation_data = pd.read_csv(path_observations, sep=";", index_col=False).drop(columns=["station"]).dropna()
    observation_data["time"] = pd.to_datetime(observation_data['time'], format='%d-%m-%Y %H:%M:%S')
    observation_data.sort_values(by='time')
    observation_data['time'] = observation_data['time'].apply(lambda x: x.replace(minute=0, second=0))
    observation_data = observation_data.groupby(['time']).apply(lambda x : x.iloc[0])
    observation_data=observation_data.rename(index=str, columns={"humidity": "humidity_Y", "precipitation": "precipitation_Y", "temperature":"temperature_Y"})
    
    return list_data, observation_data

# Associate X to previsions and Y to observations 
list_data, observation_data = get_forecast(path, path_previsions, path_observations)
X = pd.concat(list_data, axis=1)
Y = observation_data

# Delete non useful variable in prevision data
X = X.loc[:, ~X.columns.str.startswith('l')]
X = X.loc[:, ~X.columns.str.startswith('c')]
X = X.loc[:, ~X.columns.str.startswith('p')]
X = X.loc[:, ~X.columns.str.startswith('wind_dir')]
X = X.loc[:, ~X.columns.str.startswith('prec')]
X = X.loc[:, ~X.columns.str.startswith('date')]
X = X.loc[:, ~X.columns.str.startswith('time')]

# Delete non useful variable in observation data
Y = Y.drop(['date','time'], axis=1)

# Import JSON API in order to collect most recent data
import json,urllib.request
data_A = urllib.request.urlopen("http://sd-59247.dedibox.fr/agriscope/obs/transform_obs.php?STATION=Asport").read()
output_A = json.loads(data_A)

# Transform JSON data in a dataframe A
A = json.loads(data_A)
A_data = pd.DataFrame(A)
A = A_data.T

# Delelte the index of the dataframe A
A.index.name = 'time'
A.reset_index(level=0, inplace=True)

# Convert date time in correct format 
A['time'] = pd.to_datetime(A['time'])
A['time'] = pd.to_datetime(A['time'],'%d-%m-%Y %H:%M:%S')
A['time'] = A['time'].apply(lambda x: x.replace(minute=0, second=0))

# Rename dataframe A columns
A.columns = ['time', 'humidity_Y', 'precipitation_Y', 'temperature_Y']

# Reset the index of the dataframe A
A = A.set_index('time') 


# Concat vertically Y and A (Y contains former observation and A most recent one)
frames = [Y, A]
first_concat = pd.concat(frames)

# Merge X and Y dataframe in one single dataframe "df_forcast_cap"
df_forcast_cap = pd.merge(X, first_concat, right_index=True, left_index=True)
df_forcast_cap = df_forcast_cap.drop(['precipitation_Y'], axis=1)

# Delete missing values for "df_forcast_cap"
df_forcast_cap = df_forcast_cap.dropna()


Using TensorFlow backend.


In [2]:
# Pyarrow optimization for pandas/spark conversion 

#spark.conf.set("spark.sql.execution.arrow.enabled", "true") NEED TO FIX PyARROW VERSION

In [3]:
# Keep timestamps (spark dataframe deletes index)
df_forcast_cap.reset_index(drop=False, inplace=True)

In [4]:
# Convert pandas to dataframe
train_df = spark.createDataFrame(df_forcast_cap) #dataframe

In [5]:
#Compress data points into feature vector
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["temperature_Api-Agro", "wind_Api-Agro", "humidity_Api-Agro", \
                                      "humidity_Dark Sky", "temperature_Dark Sky", "wind_Dark Sky", \
                                      "humidity_Weatherbit", "temperature_Weatherbit", "wind_Weatherbit" \
                                      ],outputCol="features")
newdf = assembler.transform(train_df)

In [6]:
#Creates df with temperature_Y as label 
newdf = newdf.selectExpr("time", "temperature_Y as label", "features")

In [7]:
# Split the data into training and test sets 
newdf = newdf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("time")))

trainingData = newdf.where("rank <= .7").drop("rank")
testData = newdf.where("rank > .7").drop("rank")

In [8]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")
model = rf.fit(newdf)

In [9]:
# Make predictions.
rf_predictions = model.transform(testData)

# Select example rows to display.
rf_predictions.select("time", "prediction", "label", "features").show(10)

# Compute RMS error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)

+-------------------+------------------+-----+--------------------+
|               time|        prediction|label|            features|
+-------------------+------------------+-----+--------------------+
|2019-07-28 04:00:00| 76.62317284300812| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00| 76.62317284300812| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00| 73.79351481959176| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00| 67.34627463755837| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00| 60.65881085478945| 75.0|[22.6841473333333...|
|2019-07-28 09:00:00| 54.91606831778425| 65.0|[24.0124353333333...|
|2019-07-28 10:00:00|50.372995936193945| 56.0|[25.4114423333333...|
|2019-07-28 11:00:00|46.233539642907274| 49.0|[26.5588219999999...|
|2019-07-28 12:00:00| 43.31043530775608| 46.0|[27.6142416666666...|
|2019-07-28 13:00:00| 39.68426682886637| 43.0|[28.3597656666666...|
+-------------------+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Erro

In [10]:
# Initialize GBR
gbr = GBTRegressor(featuresCol="features", labelCol = "label", maxIter=50)

# Train model using same data structure as RandomForest
model = gbr.fit(newdf)

In [11]:
# Make predictions.
gbr_predictions = model.transform(testData)

# Display example rows
gbr_predictions.select("time", "prediction", "label", "features").show(10)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbr_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+-------------------+------------------+-----+--------------------+
|               time|        prediction|label|            features|
+-------------------+------------------+-----+--------------------+
|2019-07-28 04:00:00| 77.35031147143562| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00| 77.31141289418515| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00| 73.86210317246814| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00| 68.44148026640768| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00|  61.5352699921056| 75.0|[22.6841473333333...|
|2019-07-28 09:00:00|  60.6057524956332| 65.0|[24.0124353333333...|
|2019-07-28 10:00:00| 53.84249972178284| 56.0|[25.4114423333333...|
|2019-07-28 11:00:00| 50.56619925594794| 49.0|[26.5588219999999...|
|2019-07-28 12:00:00|  46.8183786561112| 46.0|[27.6142416666666...|
|2019-07-28 13:00:00|40.686906150113124| 43.0|[28.3597656666666...|
+-------------------+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Erro

# Layer 2

In [12]:
# Concatenate results from RF, GBR

# Isolate RF and GBR prediction columns
rf_predictions = rf_predictions.drop("features").withColumnRenamed("prediction", "rf_prediction")
gbr_predictions = gbr_predictions.drop("time","label","features").withColumnRenamed("prediction", "gbr_prediction")

concatenated_predictions = rf_predictions.join(gbr_predictions)
print(concatenated_predictions)

DataFrame[time: timestamp, label: double, rf_prediction: double, gbr_prediction: double]


In [13]:
# Compress features into feature vector
assembler = VectorAssembler(inputCols=["rf_prediction", "gbr_prediction"],outputCol="features")
ndf = assembler.transform(concatenated_predictions)

In [14]:
# Create df with time, label (temperature_Y) and feature vector
ndf = newdf.selectExpr("time", "label", "features")
ndf.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
# Split the data into training and test sets 
newdf = newdf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("time")))

trainingData = newdf.where("rank <= .7").drop("rank")
testData = newdf.where("rank > .7").drop("rank")

In [16]:
# Train RandomForest model
rf = RandomForestRegressor(featuresCol="features")
model = rf.fit(ndf)

In [17]:
# Make predictions
rf_predictions = model.transform(testData)

# Display example rows
rf_predictions.select("time","prediction", "label", "features").show(10)

# Compute RMS error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)

+-------------------+------------------+-----+--------------------+
|               time|        prediction|label|            features|
+-------------------+------------------+-----+--------------------+
|2019-07-28 04:00:00| 76.62317284300812| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00| 76.62317284300812| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00| 73.79351481959176| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00| 67.34627463755837| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00| 60.65881085478945| 75.0|[22.6841473333333...|
|2019-07-28 09:00:00| 54.91606831778425| 65.0|[24.0124353333333...|
|2019-07-28 10:00:00|50.372995936193945| 56.0|[25.4114423333333...|
|2019-07-28 11:00:00|46.233539642907274| 49.0|[26.5588219999999...|
|2019-07-28 12:00:00| 43.31043530775608| 46.0|[27.6142416666666...|
|2019-07-28 13:00:00| 39.68426682886637| 43.0|[28.3597656666666...|
+-------------------+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Erro

In [18]:
# Initialize GBR
gbr = GBTRegressor(featuresCol="features", labelCol = "label", maxIter=50)

# Train model using same data structure as RandomForest
model = gbr.fit(ndf)

In [19]:
# Make predictions.
gbr_predictions = model.transform(testData)

# Display example rows
gbr_predictions.select("prediction", "label", "features").show(10)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbr_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 77.35031147143562| 81.0|[19.7834315,9.896...|
| 77.31141289418515| 76.0|[19.8288008333333...|
| 73.86210317246814| 75.0|[20.1969648333333...|
| 68.44148026640768| 78.0|[21.2634120000000...|
|  61.5352699921056| 75.0|[22.6841473333333...|
|  60.6057524956332| 65.0|[24.0124353333333...|
| 53.84249972178284| 56.0|[25.4114423333333...|
| 50.56619925594794| 49.0|[26.5588219999999...|
|  46.8183786561112| 46.0|[27.6142416666666...|
|40.686906150113124| 43.0|[28.3597656666666...|
+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Error (RMSE) on test data = 3.99015


# Layer 3

In [20]:
# Concatenate results from RF, GBR

# Isolate RF and GBR prediction columns
rf_predictions = rf_predictions.drop("features").withColumnRenamed("prediction", "rf_prediction_2")
print(rf_predictions)
gbr_predictions = gbr_predictions.drop("time", "label","features").withColumnRenamed("prediction", "gbr_prediction_2")

# Concatenate RF and GBR with previous features
concatenated = rf_predictions.join(gbr_predictions)

DataFrame[time: timestamp, label: double, rf_prediction_2: double]


In [21]:
# Compress data points into feature vector
assembler = VectorAssembler(inputCols=["rf_prediction_2", "gbr_prediction_2"],outputCol="features")
ndf = assembler.transform(concatenated)

In [22]:
# Create df with label and feature vector
ndf = newdf.selectExpr("label", "features")
ndf.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
# Split the data into training and test sets 
newdf = newdf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("time")))

trainingData = newdf.where("rank <= .7").drop("rank")
testData = newdf.where("rank > .7").drop("rank")

In [24]:
# Train RandomForest model
rf = RandomForestRegressor(featuresCol="features")
model = rf.fit(ndf)

In [25]:
# Make predictions
rf_predictions = model.transform(testData)

# Display example rows
rf_predictions.select("time","prediction", "label", "features").show(10)

# Compute RMS error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)

+-------------------+------------------+-----+--------------------+
|               time|        prediction|label|            features|
+-------------------+------------------+-----+--------------------+
|2019-07-28 04:00:00| 76.62317284300812| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00| 76.62317284300812| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00| 73.79351481959176| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00| 67.34627463755837| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00| 60.65881085478945| 75.0|[22.6841473333333...|
|2019-07-28 09:00:00| 54.91606831778425| 65.0|[24.0124353333333...|
|2019-07-28 10:00:00|50.372995936193945| 56.0|[25.4114423333333...|
|2019-07-28 11:00:00|46.233539642907274| 49.0|[26.5588219999999...|
|2019-07-28 12:00:00| 43.31043530775608| 46.0|[27.6142416666666...|
|2019-07-28 13:00:00| 39.68426682886637| 43.0|[28.3597656666666...|
+-------------------+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Erro

In [26]:
# Initialize GBR
gbr = GBTRegressor(featuresCol="features", labelCol = "label", maxIter=50)

# Train model using same data structure as RandomForest
model = gbr.fit(ndf)

In [27]:
# Make predictions.
gbr_predictions = model.transform(testData)

# Display example rows
gbr_predictions.select("time","prediction", "label", "features").show(10)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbr_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+-------------------+------------------+-----+--------------------+
|               time|        prediction|label|            features|
+-------------------+------------------+-----+--------------------+
|2019-07-28 04:00:00| 77.35031147143562| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00| 77.31141289418515| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00| 73.86210317246814| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00| 68.44148026640768| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00|  61.5352699921056| 75.0|[22.6841473333333...|
|2019-07-28 09:00:00|  60.6057524956332| 65.0|[24.0124353333333...|
|2019-07-28 10:00:00| 53.84249972178284| 56.0|[25.4114423333333...|
|2019-07-28 11:00:00| 50.56619925594794| 49.0|[26.5588219999999...|
|2019-07-28 12:00:00|  46.8183786561112| 46.0|[27.6142416666666...|
|2019-07-28 13:00:00|40.686906150113124| 43.0|[28.3597656666666...|
+-------------------+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Erro

# Layer 4

In [28]:
# Concatenate results from RF, GBR
rf_predictions = rf_predictions.drop("features").withColumnRenamed("prediction", "rf_prediction_3")
gbr_predictions = gbr_predictions.drop("label","features").withColumnRenamed("prediction", "gbr_prediction_3")

concatenated = rf_predictions.join(gbr_predictions)

In [29]:
# Compress data points into feature vector
assembler = VectorAssembler(inputCols=["rf_prediction_3", "gbr_prediction_3"],outputCol="features")
ndf = assembler.transform(concatenated)

In [30]:
# Create df with temperature_Y and feature vector
ndf = newdf.selectExpr("label", "features")
ndf.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [31]:
# Split the data into training and test sets 
newdf = newdf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("time")))

trainingData = newdf.where("rank <= .7").drop("rank")
testData = newdf.where("rank > .7").drop("rank")

In [32]:
# Train RandomForest model
rf = RandomForestRegressor(featuresCol="features")
model = rf.fit(ndf)

In [33]:
# Make predictions
rf_predictions = model.transform(testData)

# Display example rows
rf_predictions.select("time", "prediction", "label", "features").show(10)

# Compute RMS error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_predictions)

+-------------------+------------------+-----+--------------------+
|               time|        prediction|label|            features|
+-------------------+------------------+-----+--------------------+
|2019-07-28 04:00:00| 76.62317284300812| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00| 76.62317284300812| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00| 73.79351481959176| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00| 67.34627463755837| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00| 60.65881085478945| 75.0|[22.6841473333333...|
|2019-07-28 09:00:00| 54.91606831778425| 65.0|[24.0124353333333...|
|2019-07-28 10:00:00|50.372995936193945| 56.0|[25.4114423333333...|
|2019-07-28 11:00:00|46.233539642907274| 49.0|[26.5588219999999...|
|2019-07-28 12:00:00| 43.31043530775608| 46.0|[27.6142416666666...|
|2019-07-28 13:00:00| 39.68426682886637| 43.0|[28.3597656666666...|
+-------------------+------------------+-----+--------------------+
only showing top 10 rows

Root Mean Squared Erro

In [34]:
# Initialize GBR
gbr = GBTRegressor(featuresCol="features", labelCol = "label", maxIter=50)

# Train model using same data structure as RandomForest
model = gbr.fit(ndf)

In [35]:
# Make predictions.
gbr_predictions = model.transform(testData)

# Display example rows
gbr_predictions.select("time","prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbr_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+-------------------+-----------------+-----+--------------------+
|               time|       prediction|label|            features|
+-------------------+-----------------+-----+--------------------+
|2019-07-28 04:00:00|77.35031147143562| 81.0|[19.7834315,9.896...|
|2019-07-28 05:00:00|77.31141289418515| 76.0|[19.8288008333333...|
|2019-07-28 06:00:00|73.86210317246814| 75.0|[20.1969648333333...|
|2019-07-28 07:00:00|68.44148026640768| 78.0|[21.2634120000000...|
|2019-07-28 08:00:00| 61.5352699921056| 75.0|[22.6841473333333...|
+-------------------+-----------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 3.99015
